## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_baumkataster**

## Dataset
# **Baumkataster**

## Description

Im Baumkataster werden alle städtischen Bäume im Strassenraum erfasst, welche von Grün Stadt Zürich verwaltet oder gepflegt werden. Ergänzt wird das Baumkataster durch das Obstbauminventar sowie Bäume ausgewählter öffentlicher Grünanlagen und private Bäume.

**Zweck**: Das Baumkataster dient der Verwaltung der Bäume im öffentlichen Raum. Es ist Hilfsmittel für die Pflege dieser Bäume und für den Ersatz &quot;alter&quot; Bäume. Zudem ist es eine wichtige Informationsgrundlage für Planende, Projektierende die sich über die bestehenden Baumstandorte orientieren wollen. Interessierte sollen jederzeit aktuell und präzise über Baumarten und Baumstandorte informieren können.

**Genereller Hinweis zum Geodatensatz:**

Es handelt sich hierbei um einen Geodatensatz, welcher aus mehreren Geodatenelmenten besteht. Diese Geodatenelemente werden hier nicht im Detail beschrieben. Sie finden jedoch sämtliche Informationen - wie beispielsweise die Attributbeschreibungen - [«**Baumkataster»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/5b17c52e-1fcd-e0e4-47b2-ab784f1bead7).
			   

**Informationen zum Datensatz:**

Im Baumkataster werden alle städtischen Bäume im Strassenraum erfasst, welche von Grün Stadt Zürich verwaltet oder gepflegt werden. Ergänzt wird das Baumkataster durch das Obstbauminventar sowie Bäume ausgewählter öffentlicher Grünanlagen und private Bäume. Hauptauslöser von Mutationen ist der alljährliche Baumersatz der Strassenbäume. Ebenfalls werden in den Grünanlagen alle Zustandserfassungen und Pflegemassnahmen erfasst. Eine flächendeckende Erfassung der Bäume in Grünanlagen ist zwar angedacht, wird aber schrittweise umgesetzt. Der Baumkataster wird der Öffentlichkeit über eine WebGIS-Anwendung zur Verfügung gestellt.

**Datenerfassung:**

Das Baumkataster wurde basierend auf analogen Plänen digitalisiert. Seit 2001 erfolgt die Vermessung der Baumstandorte tachymetrisch. Die Standortgenauigkeit des Stammmittelpunktes beträgt <1cm bis 30cm für tachymetrisch eingemessene Baumstandorte. Private Bäume und Bäume in Grünanlagen sind meist mit einer Genauigkeit von 0.5 bis 2m erfasst.

**Datengrundlage:**

Digitalisier-Grudlage, Berechnungs-Grudlage (z.B Naturwertindex)

**Geodatenelemente:**

Die vollständige Metadatenbeschreibung der Geodatenelemente finden Sie ebenfalls unter [«**Baumkataster»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/md.viewer#/full_view/5b17c52e-1fcd-e0e4-47b2-ab784f1bead7).

***BAUMKATASTER_BAUMSTANDORTE***: 

Baumstandorte in der Stadt Zürich


***BAUMKATASTER_KRONENDURCHMESSER***: 

Darstellung des Kronendurchmessers für den CAD-Export. Die Kronendurchmesser im Baumkataster sind unvollständig, werden unregelmässig nachgeführt und sind nur geschätzt. Bei Bäumen bei denen kein Kronendurchmesser vorhanden ist, wird ein Durchmesser von 8m verwendet.


***BAUMSTANDORTE_K***: 

Darstellung des Baumstandortes als Kreis (10cm Radius) für den CAD-Export.




**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/5b17c52e-1fcd-e0e4-47b2-ab784f1bead7.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_baumkataster)

https://www.stadt-zuerich.ch/geodaten/download/Baumkataster?format=geojson_link<br>


## Metadata
- **Publisher** `Region Nord, Grün Stadt Zürich, Tiefbau- und Entsorgungsdepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Umwelt`
- **Tags** `['baum', 'geodaten', 'geoportal', 'obstbaum', 'polygondaten', 'punktdaten', 'stassenbaum', 'stzh', 'vektordaten']`
- **Metadata_created** `2023-11-06T03:06:58.988791`
- **Metadata_modified** `2025-10-11T06:31:53.604862`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Baumkataster?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch